In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json
from os import path
import os
from urllib import parse

In [3]:
def load_json(json_fname):
    with open(json_fname,encoding="utf8") as fp:
        return json.load(fp)

# save link to file

In [179]:
title_key=lambda title:re.sub("\W","",title.lower(),)

In [182]:
{
    "title_key":{
        "title":str,
        "author":[
            str,
            str,
        ],
        "file_name":str,
        "conf":str
    }
}

{'title_key': {'title': str,
  'author': [str, str],
  'file_name': str,
  'conf': str}}

In [176]:
def parse_eccv2018():
    conf="ECCV2018"
    assert isinstance(conf,str)
    page=requests.get(f"http://openaccess.thecvf.com/{conf}.py")
    assert page.ok
    soup=BeautifulSoup(page.content,"lxml")
    dl=soup.select_one("dl")

    title_list=[a.text for a in  dl.select("dt > a")]

    authors_list=[[au.attrs["value"] for au in dd.select("form > input")]  for dd in dl.select("dt + dd")]

    dd=dl.select("dt + dd")[0]

    link_list=[ parse.urljoin("http://openaccess.thecvf.com",a.attrs["href"]) for a in dl.select("dd + dd > a")]
    file_name_list=[path.split(link)[-1] for link in link_list ]
    ll=map(lambda t,a,l,fn:dict(title=t,authors=a,link=l,conf=conf,file_name=fn) , title_list,authors_list,link_list,file_name_list)
    ll=list(ll)
    
    return ll

In [177]:
ll=[]
# ll+=parse("ECCV2016")
ll+=parse_eccv2018()

In [168]:
ll.append({
    "title": "Summarizing First-Person Videos from Third Persons' Points of Views",
    "authors": [
      "HSUAN-I HO",
      "Wei-Chen Chiu",
      "Yu-Chiang Frank Wang"
    ],
    "link": "http://openaccess.thecvf.com/content_ECCV_2018/papers/HSUAN-I_HO_Summarizing_First-Person_Videos_ECCV_2018_paper.pdf",
    "file_name":"HSUAN-I_HO_Summarizing_First-Person_Videos_ECCV_2018_paper.pdf",
    "conf": "ECCV2018"}
  )

In [169]:
with open("links.json","w") as fp:
    json.dump({"l":ll},fp,indent=2)

# 下载

In [4]:
if not path.exists("full_text"):
    os.mkdir("full_text")

In [22]:
def download(link,fpath="full_text"):
    try:
        res=requests.get(link["link"],proxies={"http":"127.0.0.1:10800"})
        if res.ok:
            with open(path.join(fpath,link["file_name"]),"wb") as fp:
                fp.write(res.content)
        return link
    except Exception as e:
        print(f"{link} {e}")
        raise

In [23]:
from multiprocessing.dummy import Pool

In [24]:
ll=load_json("links.json")["l"]

Boosted Attention: Leveraging Human Attention for Image Captioning
Video Object Segmentation with Joint Re-identification and Attention-Aware Mask Propagation
Making Deep Heatmaps Robust to Partial Occlusions for 3D Object Pose Estimation
Snap Angle Prediction for 360° Panoramas
Incremental Multi-graph Matching via Diversity and Randomness based Graph Clustering
End-to-End Joint Semantic Segmentation of Actors and Actions in Video
C-WSL: Count-guided Weakly Supervised Localization
Deep Co-Training for Semi-Supervised Image Recognition
Scalable Exemplar-based Subspace Clustering on Class-Imbalanced Data
Multi-view to Novel view: Synthesizing novel views with Self-Learned Confidence
Pixel2Mesh: Generating 3D Mesh Models from Single RGB Images
Visual Text Correction
Progressive Neural Architecture Search


In [ ]:
def download_handler(link):
    print(link["title"])
    link["downloaded"]=True

pool=Pool(16)
for link in ll:
    if not link.get("downloaded",False):
        pool.apply_async(download,(link,),callback=download_handler,)
pool.close()
pool.join()

RCAA: Relational Context-Aware Agents for Person Search
Visual Coreference Resolution in Visual Dialog using Neural Module Networks
Learning to Segment via Cut-and-Paste
Dynamic Conditional Networks for Few-Shot Learning
Deep Factorised Inverse-Sketching
Learning Blind Video Temporal Consistency
CNN-PS: CNN-based Photometric Stereo for General Non-Convex Surfaces
Dense Pose Transfer
Separating Reflection and Transmission Images in the Wild
Diverse Image-to-Image Translation via Disentangled Representations
Product Quantization Network for Fast Image Retrieval
CBAM: Convolutional Block Attention Module
Learning-based Video Motion Magnification
Semi-convolutional Operators for Instance Segmentation
Skeleton-Based Action Recognition with Spatial Reasoning and Temporal Stack Learning
Learnable PINs: Cross-Modal Embeddings for Person Identity
Video Object Segmentation with Joint Re-identification and Attention-Aware Mask Propagation
Hand Pose Estimation via Latent 2.5D Heatmap Regression
Sw

In [ ]:
with open("links.json","w") as fp:
    json.dump({"l":ll},fp,indent=2)

# 存到映射

In [192]:
links_map=dict()
for link in ll:
    links_map[title_key(link["title"])]=link
with open("links_map.json","w") as fp:
    json.dump(links_map,fp,indent=2)